In [1]:
import pandas as pd
#import tensorflow as tf  #tf2.0
import tensorflow.compat.v1 as tf
import os, io
from PIL import Image
from object_detection.utils import dataset_util

In [2]:
#Creates a TF train example
from ast import literal_eval
def create_tf_example(group, path, ext):
    
    #hardcoded class text and class index for single class detector
    class_text = 'wheat'.encode('utf8')
    class_idx = 1
    
    fname = os.path.join(path, '{}{}'.format(group.filename, ext))
    #with tf.io.gfile.GFile(fname, 'rb') as fid:  #tf2.0
    with tf.gfile.GFile(fname, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size
    
    
    filename = fname.encode('utf8')
    image_format = ext.encode('utf8')
    #print(filename)
    #print(width, ' ', height)
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []
    
    for index, row in group.object.iterrows():
        #print( index, ':\n', row)
        bbox = literal_eval(row['bbox'])
        xmins.append(bbox[0] / width)
        xmaxs.append((bbox[0]+bbox[2]) / width )
        ymins.append(bbox[1] / height)
        ymaxs.append((bbox[1]+bbox[3])/ height )
        classes_text.append(class_text)
        classes.append(class_idx)
        
    #for x1, x2, y1, y2 in zip(xmins, xmaxs, ymins, ymaxs):
    #    print(x1, x2, y1, y2)
    #input('bboxes')
    
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    #print(tf_example)
    #input('tf example')
    return tf_example

In [3]:
#Read data from csv to pandas dataframe
data_foldername = '/home/manju/data/ML/kaggle/wheat-detection'
train_subfoldername = 'train'
train_csv_filename = os.path.join(data_foldername, 'train.csv')
img_ext = '.jpg'

tf_record_filename = '/home/manju/data/ML/training_workspace/kaggle-wheat-detection/annotations/train.record'

train_foldername = os.path.join(data_foldername, train_subfoldername)
print(train_csv_filename)
df = pd.read_csv(train_csv_filename)
print(df)
print(df.values)
print(df.dtypes)
print(df.head())

/home/manju/data/ML/kaggle/wheat-detection/train.csv
         image_id  width  height                         bbox     source
0       b6ab77fd7   1024    1024   [834.0, 222.0, 56.0, 36.0]    usask_1
1       b6ab77fd7   1024    1024  [226.0, 548.0, 130.0, 58.0]    usask_1
2       b6ab77fd7   1024    1024  [377.0, 504.0, 74.0, 160.0]    usask_1
3       b6ab77fd7   1024    1024  [834.0, 95.0, 109.0, 107.0]    usask_1
4       b6ab77fd7   1024    1024  [26.0, 144.0, 124.0, 117.0]    usask_1
...           ...    ...     ...                          ...        ...
147788  5e0747034   1024    1024    [64.0, 619.0, 84.0, 95.0]  arvalis_2
147789  5e0747034   1024    1024  [292.0, 549.0, 107.0, 82.0]  arvalis_2
147790  5e0747034   1024    1024  [134.0, 228.0, 141.0, 71.0]  arvalis_2
147791  5e0747034   1024    1024   [430.0, 13.0, 184.0, 79.0]  arvalis_2
147792  5e0747034   1024    1024   [875.0, 740.0, 94.0, 61.0]  arvalis_2

[147793 rows x 5 columns]
[['b6ab77fd7' 1024 1024 '[834.0, 222.0, 56.0

In [4]:
#Group all data points by filename
from collections import namedtuple
data = namedtuple('data', ['filename','object'])
gb = df.groupby('image_id')
grouped_data = [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]
#print(len(gb.groups))
print(len(grouped_data))
print(grouped_data[1].filename)
print(grouped_data[1].object)
#for d in grouped_data:
#    print(d.filename)
#    print(d.object)

3373
005b0d8bb
       image_id  width  height                          bbox   source
1406  005b0d8bb   1024    1024   [765.0, 879.0, 116.0, 79.0]  usask_1
1407  005b0d8bb   1024    1024    [84.0, 539.0, 153.0, 96.0]  usask_1
1408  005b0d8bb   1024    1024    [855.0, 781.0, 96.0, 76.0]  usask_1
1409  005b0d8bb   1024    1024   [798.0, 934.0, 133.0, 88.0]  usask_1
1410  005b0d8bb   1024    1024  [330.0, 395.0, 145.0, 215.0]  usask_1
1411  005b0d8bb   1024    1024       [0.0, 0.0, 72.0, 106.0]  usask_1
1412  005b0d8bb   1024    1024    [711.0, 60.0, 138.0, 97.0]  usask_1
1413  005b0d8bb   1024    1024  [157.0, 393.0, 220.0, 108.0]  usask_1
1414  005b0d8bb   1024    1024   [967.0, 545.0, 57.0, 139.0]  usask_1
1415  005b0d8bb   1024    1024  [305.0, 577.0, 220.0, 103.0]  usask_1
1416  005b0d8bb   1024    1024  [562.0, 518.0, 194.0, 110.0]  usask_1
1417  005b0d8bb   1024    1024  [737.0, 555.0, 123.0, 181.0]  usask_1
1418  005b0d8bb   1024    1024  [212.0, 151.0, 184.0, 116.0]  usask_1
1419 

In [5]:
#Create TF record file and write each example into it
#tfr_writer = tf.io.TFRecordWriter( tf_record_filename) #tf2.0
tfr_writer = tf.python_io.TFRecordWriter( tf_record_filename)
for group in grouped_data:
    tf_example = create_tf_example(group, train_foldername, img_ext)
    tfr_writer.write(tf_example.SerializeToString())
tfr_writer.close()